In [1]:
import numpy as np
import pandas as pd

# @software{dias2019fuzzy,
#   author       = {Madson Luiz Dantas Dias},
#   title        = {fuzzy-c-means: An implementation of Fuzzy $C$-means clustering algorithm.},
#   month        = may,
#   year         = 2019,
#   publisher    = {Zenodo},
#   doi          = {10.5281/zenodo.3066222},
#   url          = {https://git.io/fuzzy-c-means}
# }
from fcmeans import FCM

from sklearn.decomposition import PCA
from tqdm import tqdm

import math

In [2]:
np.set_printoptions(precision=4, suppress=True)

# Input data

We still use the example in Figure 2. If the input data has a large column dimansion, some dimensional reducing techniques can be applied.

In [3]:
patterns = [['A', 'B'], 
            ['A', 'B', 'C'], 
            ['A', 'B', 'D'], 
            ['A', 'B', 'C', 'D'], 
            ['A', 'C', 'D'], 
            ['B', 'C'], 
            ['B', 'C', 'D'], 
            ['B', 'C', 'E'], 
            ['B', 'D'], 
            ['B', 'E'], 
            ['C', 'D']]
# The conditional co-occurrence matrix
CM = np.array([[  1, 4/5, 3/5, 3/5,   0], 
               [1/2,   1, 5/8, 1/2, 1/4], 
               [3/7, 5/7,   1, 4/7, 1/7], 
               [1/2, 2/3, 2/3,   1,   0], 
               [  0,   1, 1/2,   0,   1]])

n_features = CM.shape[0]
feature_index = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}

In [4]:
# pca_init = PCA(n_components='mle')
# CM = pca_init.fit_transform(CM)

In [5]:
print(CM.shape)

(5, 5)


# Step 1: Multi-granularity fuzzy clustering embedding  (Section IV. B)

### Multi-granularity fuzzy clustering

The function below returns a clustering indicator matrix (Def.11) according to the parameter n_clusters for fuzzy c-means clustering (FCM).

In [6]:
def calc_indicator_matrix(X, n_clusters):
    # Execute a fuzzy c-means clustering.
    fcm = FCM(n_clusters=n_clusters)
    fcm.fit(X)
    
    # Return the indicator matrix.
    return fcm.u

Then, we execute the fuzzy clustering with different number of clusters, which we called multi-granularity. The indicator matrices are collected and concatenated horizontally.

In [7]:
max_clusters = n_features - 1

def remove_small_clusters(indicator_matrix, n_clusters):
    # Calculate the sum of u_ij.
    membership_sum = np.sum(indicator_matrix, axis=0)
    # Check whether is in SC.
    is_small = membership_sum <= math.ceil(n_features / n_clusters)

    return np.delete(indicator_matrix, is_small, axis=1), np.sum(is_small)

alpha = 1
small_count = 0
indicator_matrices = []
for n_clusters in tqdm(range(2, max_clusters + 1)):
    # Line 5 in Algorithm 2.
    indicator_matrix = calc_indicator_matrix(CM, n_clusters)
    # Lines 6-7 in Algorithm 2.
    indicator_matrix, sc = remove_small_clusters(indicator_matrix, n_clusters)
    # Line 8 in Algorithm 2.
    indicator_matrices.append(indicator_matrix)
    
    # Check the quit condition. Line 9-12 in Algorithm 2.
    small_count += sc
    if small_count >= math.ceil(n_clusters / alpha):
        break

# The variable is named IM in the paper.
concated_indicator_matrix = np.hstack(indicator_matrices)

 33%|██████████████▋                             | 1/3 [00:00<00:00, 116.50it/s]


In this case, the concatenated indicator matrix (i.e., $IM$) is shown as below:

In [8]:
print(concated_indicator_matrix)

[[0.9268 0.0246]
 [0.8947 0.628 ]
 [0.9217 0.8759]
 [0.9402 0.6342]
 [0.0006 0.0001]]


Each column of the $IM$ represents affiliation degree (or called membership degree) to a cluster. Each cluster can be regarded as a semantic aspect.

### PCA
Finally, we try to execute a PCA on the resulted $IM$. That's because the coupling relations are not only exist among features but also semantic aspects. The PCA with a full number of components can be regarded as a change of basis, and independent semantics are captured. Moreover, PCA can be utilized to reduce dimensionalities, further refine the embedding vectors.

In [9]:
centered_concated_indicator_matrix = concated_indicator_matrix - np.mean(concated_indicator_matrix, axis=0)

In [10]:
pca = PCA() # PCA(n_components=) for dimensional reducing.
EM = pca.fit_transform(centered_concated_indicator_matrix)

The embeddings are listed as below. Each column can be treated as an independent but principle semantic aspect.

In [11]:
print(EM)

[[ 0.1407  0.4275]
 [-0.2489 -0.0344]
 [-0.4383 -0.1966]
 [-0.2863 -0.0078]
 [ 0.8328 -0.1887]]


### Markov's inequality for dimensional reducing

In [12]:
eps = 0.8

1. Calculate distancs to other columns.
2. Calculate average distances to others.
3. Remove the clusters whose $maximum\_distances$ is less than $\frac{1}{1-\epsilon}*average\_distances$ (Markov's inquality).

In [13]:
while True:
    if EM.shape[1] <= 1:
        break
    
    exit_flag = True
    for i in range(EM.shape[1]):
        embedding = EM[:, i]
        embedding_minus = EM - embedding[:, np.newaxis]
        # Calculate distances to other columns.
        distances = np.sqrt(np.sum(np.square(embedding_minus), axis=0))

        maximum_distance = np.max(distances)
        average_distance = np.sum(distances) / (EM.shape[1] - 1)
        # Markov's inequality for dimensional reducing
        if maximum_distance < 1 / (1-eps)*average_distance:
            EM = np.delete(EM, i, axis=1)
            exit_flag = False
            break
    if exit_flag:
        break 

The final EM is:

In [14]:
print(EM)

[[ 0.4275]
 [-0.0344]
 [-0.1966]
 [-0.0078]
 [-0.1887]]


# Step 2: Sampling and interaction (Section III. B)

### Sampling

Each sample has 3 patterns. The parameter $\mu$ (Def. 5) is set to 0.4.

In [15]:
candidate_patterns = patterns.copy()

In [16]:
sample_size = 2
mu = 0.4

In [17]:
# Def.2
def calc_info(sample):
    numerator_feature = set()
    for p in sample:
        for f in p:
            numerator_feature.add(f)
    
    return len(numerator_feature) / n_features

# Def.3
def calc_local_info(sample):
    return calc_info(sample)
    
# Def.4
def calc_global_info(sample, history_samples):
    return calc_info(sample + history_samples) 

In [18]:
history_samples = []
def sample(candidate_patterns, sample_size=2, mu=0.5, verbose=True):
    sample = []
    if len(candidate_patterns) <= sample_size:
        sample = candidate_patterns.copy()
        candidate_patterns.clear()
        return sample
    
    while len(sample) < sample_size:
        best_info = 0
        for idx, pattern in enumerate(candidate_patterns):
            local_info = calc_local_info(sample + pattern)
            global_info = calc_global_info(sample + pattern, history_samples)
            info = mu * local_info + (1 - mu) * global_info
            if verbose:
                print("pattern: ", pattern, ", local info = %.3f, global info = %.3f, info = %.3f" % (local_info, global_info, info))
            if info > best_info:
                best_info = info
                best_pattern = pattern

        sample.append(best_pattern)
        history_samples.append(best_pattern)
        candidate_patterns.remove(best_pattern)
        if verbose:
            print("-> pattern: ", best_pattern, " is sampled!")
            print("-"*80)
        
    return sample

In [19]:
sampled_patterns = sample(candidate_patterns)
print("* Sample #1: ", sampled_patterns)

pattern:  ['A', 'B'] , local info = 0.400, global info = 0.400, info = 0.400
pattern:  ['A', 'B', 'C'] , local info = 0.600, global info = 0.600, info = 0.600
pattern:  ['A', 'B', 'D'] , local info = 0.600, global info = 0.600, info = 0.600
pattern:  ['A', 'B', 'C', 'D'] , local info = 0.800, global info = 0.800, info = 0.800
pattern:  ['A', 'C', 'D'] , local info = 0.600, global info = 0.600, info = 0.600
pattern:  ['B', 'C'] , local info = 0.400, global info = 0.400, info = 0.400
pattern:  ['B', 'C', 'D'] , local info = 0.600, global info = 0.600, info = 0.600
pattern:  ['B', 'C', 'E'] , local info = 0.600, global info = 0.600, info = 0.600
pattern:  ['B', 'D'] , local info = 0.400, global info = 0.400, info = 0.400
pattern:  ['B', 'E'] , local info = 0.400, global info = 0.400, info = 0.400
pattern:  ['C', 'D'] , local info = 0.400, global info = 0.400, info = 0.400
-> pattern:  ['A', 'B', 'C', 'D']  is sampled!
-----------------------------------------------------------------------

### Interaction

After we determined the sample, interactions are executed, collecting your favourites in the sample.

In [20]:
def interact(sampled_patterns, verbose=True):
    preferred_patterns = []
    for pattern in sampled_patterns:
        feedback = input("Do you like the co-location pattern: ["+ ', '.join(pattern) +"]? (Y/n)\n")
        if feedback.lower() == 'n':
            continue
        else:
            if verbose:
                print(pattern, " is your favourite.")
            preferred_patterns.append(pattern)
    return preferred_patterns

preferred_patterns = interact(sampled_patterns)

Do you like the co-location pattern: [A, B, C, D]? (Y/n)
y
['A', 'B', 'C', 'D']  is your favourite.
Do you like the co-location pattern: [B, C, E]? (Y/n)
n


# Step 3: Preference selection (Section IV. C)

Define the pattern semantic distance (Def.14).

In [21]:
def min_dist_operator(pattern1_embedding, pattern2_embedding):
    pattern1_embedding_tiled = pattern1_embedding[: , np.newaxis, :]
    pattern1_embedding_tiled = np.repeat(pattern1_embedding_tiled, pattern2_embedding.shape[0], axis=1)
    pattern1_embedding_minus = pattern1_embedding_tiled - pattern2_embedding
    distances = np.sqrt((pattern1_embedding_minus * pattern1_embedding_minus).sum(axis=-1))
    min_distances = np.min(distances, axis=-1)

    return np.sum(min_distances)

In [22]:
def calc_semantic_dist(pattern1, pattern2):
    pattern1_embedding = np.array(list(map(lambda f : EM[feature_index[f]], pattern1)))
    pattern2_embedding = np.array(list(map(lambda f : EM[feature_index[f]], pattern2)))
    
    distance1 = min_dist_operator(pattern1_embedding, pattern2_embedding)
    distance2 = min_dist_operator(pattern2_embedding, pattern1_embedding)
    
    return (distance1 + distance2) / 2

Define the pattern structural distance (Def.13).

In [23]:
def calc_structural_dist(pattern1, pattern2, adjusting_factor=1e-6):
    pattern1_set = set(pattern1)
    pattern2_set = set(pattern2)
    
    intersection = pattern1_set.intersection(pattern2)
    union = pattern1_set.union(pattern2)
    
    return 1 - len(intersection) / len(union) + adjusting_factor

Define the final pattern distance (Def.15).

In [24]:
def calc_pattern_distance(pattern1, pattern2, structural_influence_index=1.0):
    semantic_distance = calc_semantic_dist(pattern1, pattern2)
    structural_distance = calc_structural_dist(pattern1, pattern2, 0)
    
    return semantic_distance / math.pow(structural_distance, structural_influence_index)

Let's recall what we have selected as preferences before:

In [25]:
print(preferred_patterns)

[['A', 'B', 'C', 'D']]


For each preferred co-location patterns, we choose the top-2 nearest patterns as deduced preferences.

In [26]:
def select_preference(preferred_patterns, candidate_patterns, 
                      nearest_pattern_size=2, structural_influence_index=1.0, verbose=True):
    selected_preferences = []
    for preferred_pattern in preferred_patterns:
        pattern_dist_pairs = []
        for candidate_pattern in candidate_patterns:
            distance = calc_pattern_distance(preferred_pattern, candidate_pattern)
            pattern_dist_pairs.append([distance, candidate_pattern])

        pattern_dist_pairs = sorted(pattern_dist_pairs)

        # Select the top-k nearest patterns.
        top_nearest_patterns = pattern_dist_pairs[: nearest_pattern_size]

        if verbose:
            print('For the co-location pattern ', preferred_pattern, ", we choose these patterns as your preferences:")
        for idx, top_nearest_pattern in enumerate(top_nearest_patterns):
            if verbose:
                print('%d. %s' % (idx + 1, top_nearest_pattern[1]))
            selected_preferences.append(top_nearest_pattern[1])
            candidate_patterns.remove(top_nearest_pattern[1])
        if verbose:
            print('-'*80)
    return selected_preferences
        
# The variable ns.
nearest_pattern_size = 2
# The variable w in the paper.
structural_influence_index = 1.0

selected_preferences = select_preference(preferred_patterns, candidate_patterns, nearest_pattern_size, structural_influence_index)

For the co-location pattern  ['A', 'B', 'C', 'D'] , we choose these patterns as your preferences:
1. ['A', 'B', 'C']
2. ['A', 'C', 'D']
--------------------------------------------------------------------------------


Our recommendations for this round are:

In [27]:
print(selected_preferences)

[['A', 'B', 'C'], ['A', 'C', 'D']]


# Further steps

After providing our recommendations, the round is over. Next, the algorithm, FITTER, iteratively executes Steps 2 and 3, until the vairable $candidate\_patterns$ is empty.

In [28]:
while len(candidate_patterns) != 0:
    sampled_patterns = sample(candidate_patterns, verbose=False)
    preferred_patterns = interact(sampled_patterns, verbose=False)
    round_preferences = select_preference(preferred_patterns, candidate_patterns, verbose=False)
    if len(round_preferences) != 0:
        selected_preferences.extend(round_preferences)

Do you like the co-location pattern: [A, B, D]? (Y/n)
y
Do you like the co-location pattern: [B, C]? (Y/n)
y
Do you like the co-location pattern: [B, D]? (Y/n)
n


The final results are:

In [29]:
print(selected_preferences)

[['A', 'B', 'C'], ['A', 'C', 'D'], ['A', 'B'], ['C', 'D'], ['B', 'E'], ['B', 'C', 'D']]
